In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

Chain of Density: https://arxiv.org/pdf/2309.04269.pdf


In [2]:
import time
import textwrap

from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain.document_loaders import WebBaseLoader
from langchain.schema.runnable import RunnablePassthrough

# Load some data to summarize
loader = WebBaseLoader("https://teddylee777.github.io/data-science/optuna/")
docs = loader.load()
content = docs[0].page_content

# Get this prompt template
prompt = hub.pull("lawwu/chain_of_density")

# The chat model output is a JSON list of dicts, with SimpleJsonOutputParser
# we can convert it o a dict, and it suppors streaming.
json_parser = SimpleJsonOutputParser()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
chain = (
    {"ARTICLE": RunnablePassthrough()}
    | prompt
    | ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0.1)
    | json_parser
)

In [4]:
chain.invoke(content)

[{'Missing_Entities': 'Optuna',
  'Denser_Summary': "Optuna is a hyperparameter optimization framework; it offers suggest_categorical, suggest_int, and suggest_float methods. Optuna's objective is to minimize the error using 5-Fold Cross Validation."},
 {'Missing_Entities': 'objective function',
  'Denser_Summary': 'The objective function in Optuna aims to minimize error through 5-Fold Cross Validation. It utilizes suggest_categorical, suggest_int, and suggest_float methods for parameter optimization.'},
 {'Missing_Entities': 'RandomForestRegressor',
  'Denser_Summary': 'The RandomForestRegressor model is used in the Optuna optimization process. The objective function minimizes error using 5-Fold Cross Validation with RandomForestRegressor.'},
 {'Missing_Entities': 'study.best_params',
  'Denser_Summary': 'The best_params from the study in Optuna are printed. The RandomForestRegressor model is instantiated with the best_params for training.'},
 {'Missing_Entities': 'mean_squared_error'

In [5]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.base import BaseCallbackHandler
import json


# Load some data to summarize
loader = WebBaseLoader(
    "https://www.aitimes.com/news/articleView.html?idxno=131777")
docs = loader.load()
content = docs[0].page_content
# Load the prompt
# prompt = hub.pull("langchain-ai/chain-of-density:4f55305e")


class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs):
        print(token, end="", flush=True)


prompt = ChatPromptTemplate.from_template(
    """Article: {ARTICLE}
You will generate increasingly concise, entity-dense summaries of the above article. 

Repeat the following 2 steps 5 times. 

Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. 
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. 

A missing entity is:
- relevant to the main story, 
- specific yet concise (50 words or fewer), 
- novel (not in the previous summary), 
- faithful (present in the article), 
- anywhere (can be located anywhere in the article).

Guidelines:

- The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~200 words.
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article. 
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities. 

Remember, use the exact same number of words for each summary.
Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
Use only KOREAN language to reply."""
)


# Create the chain, including
chain = (
    prompt
    | ChatOpenAI(
        temperature=0,
        model="gpt-4-turbo-preview",
        streaming=True,
        callbacks=[StreamCallback()],
    )
    | JsonOutputParser()
    | (lambda x: x[-1]["Denser_Summary"])
)

# Invoke the chain
result = chain.invoke({"ARTICLE": content})
print(result)

```json
[
    {
        "Missing_Entities": "",
        "Denser_Summary": "이 기사는 데이터사이언스, 머신러닝, 인공지능에 대한 개념을 설명하고 있습니다. 박정현 서울대 EPM 연구원이 작성한 이 글에서는 이 세 분야가 우리 생활에 어떻게 적용되고 있는지, 그리고 이 분야들을 공부하고자 하는 사람들을 위한 가이드를 제공하고 있습니다. 또한, 데이터사이언스가 가장 넓은 범위를 가지고 있으며, 머신러닝과 인공지능이 어떻게 서로 다른지에 대해서도 설명하고 있습니다. 이 글은 이 분야에 대한 기본적인 이해를 돕기 위해 여러 대학에서 데이터사이언스와 인공지능 대학원을 설립하는 등의 교육적 변화에 대해서도 언급하고 있습니다."
    },
    {
        "Missing_Entities": "튜링테스트; 지도학습(supervised learning); 비지도 학습(unsupervised learning)",
        "Denser_Summary": "박정현 서울대 EPM 연구원이 작성한 이 글에서는 데이터사이언스, 머신러닝, 인공지능의 개념과 실생활 적용 사례를 설명합니다. 데이터사이언스가 가장 넓은 범위를 가지며, 인공지능의 이해를 위해 튜링테스트를 언급하고, 머신러닝을 지도학습과 비지도 학습으로 구분하여 설명합니다. 또한, 이 분야의 교육적 변화와 대학에서의 데이터사이언스 및 인공지능 대학원 설립에 대해서도 언급하며, 이 분야에 대한 기본적인 이해를 돕습니다."
    },
    {
        "Missing_Entities": "알파고; GPT-3; 빅데이터; 통계적 기법",
        "Denser_Summary": "박정현 서울대 EPM 연구원이 작성한 글에서는 데이터사이언스, 머신러닝, 인공지능의 정의와 알파고, GPT-3 같은 실생활 적용 사례를 소개합니다. 데이터사이언스는 빅데이터와 통계적 기법을 기반으로 하며, 인공지능의 이해를 위한 튜링테스트, 머신러닝의 지도학습 및 비지